In [1]:
import os, json, ast
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [4]:
# création d'un dataframe à partir du csv de données
job_events_df = pd.read_csv("../data/raw_unmerged/job_events_18-inf.csv")
job_events_df_sup = pd.read_csv("../data/raw_unmerged/job_events_18-sup.csv")
job_events_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
job_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51566 entries, 0 to 51565
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           51566 non-null  int64 
 1   payload      51566 non-null  object
 2   received_at  51566 non-null  object
 3   machine_id   51566 non-null  int64 
 4   tag          51566 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [5]:
job_events_df_sup.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
job_events_df_sup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60539 entries, 0 to 60538
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           60539 non-null  int64 
 1   payload      60539 non-null  object
 2   received_at  60539 non-null  object
 3   machine_id   60539 non-null  int64 
 4   tag          60539 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [6]:
job_events_merged_df = pd.concat([job_events_df, job_events_df_sup], ignore_index=True)
job_events_merged_df = job_events_merged_df.sort_values(by='received_at')

# Vérifier la fusion
job_events_merged_df.head()

,id,payload,received_at,machine_id,tag
51160,82917,"{""iper"": [{""id"": ""PRINT_ENGINE_1"", ""LED"": 50, ...",2022-02-22 09:43:18.114000,18,job-started
51161,82918,"{""path"": ""D:/IMAGES/Standard/1504750#1/0000001...",2022-02-22 09:43:18.290000,18,job-preview-ready
51162,82919,"{""jobId"": ""1645522997"", ""jobState"": ""SUCCESS"",...",2022-02-22 09:44:33.472000,18,job-ended
51163,82921,"{""iper"": [{""id"": ""PRINT_ENGINE_1"", ""LED"": 50, ...",2022-02-22 09:45:01.297000,18,job-started
51164,82922,"{""path"": ""D:/IMAGES/Standard/1504749#1/0000001...",2022-02-22 09:45:01.456000,18,job-preview-ready


In [7]:
job_events_merged_df.tail()

,id,payload,received_at,machine_id,tag
79292,873661,"{""iper"": [{""id"": ""PRINT_ENGINE_1"", ""LED"": 50, ...",2023-11-16 12:48:08.191000,18,job-started
79293,873662,"{""path"": ""D:/IMAGES/Standard/2118611#1/0000001...",2023-11-16 12:48:08.328000,18,job-preview-ready
79294,873666,"{""jobId"": ""1700138888"", ""jobState"": ""SUCCESS"",...",2023-11-16 12:51:59.168000,18,job-ended
79295,873671,"{""iper"": [{""id"": ""PRINT_ENGINE_1"", ""LED"": 50, ...",2023-11-16 12:53:08.804000,18,job-started
79296,873672,"{""path"": ""D:/IMAGES/Standard/2118611#1/0000001...",2023-11-16 12:53:08.928000,18,job-preview-ready


In [8]:
print(len(job_events_merged_df))  

112105


In [9]:
job_events_merged_df.to_csv("../data/raw/job_events.csv", index=False)

In [10]:
metrics_merged_df = pd.read_csv("../data/raw/metrics.csv")
job_events_merged_df = pd.read_csv("../data/raw/job_events.csv")
jobs_merged_df = pd.read_csv("../data/raw/jobs.csv")

C:\Users\Allan\AppData\Local\Temp\ipykernel_6832\3270355420.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs_merged_df = pd.read_csv("../data/raw/jobs.csv")


In [11]:
# Convertir les colonnes de date
metrics_merged_df['created_at'] = pd.to_datetime(metrics_merged_df['created_at'])
job_events_merged_df['received_at'] = pd.to_datetime(job_events_merged_df['received_at'])
jobs_merged_df['started_at'] = pd.to_datetime(jobs_merged_df['started_at'])

# Trouver et afficher les valeurs maximales et minimales pour chaque DataFrame
max_date_metrics = metrics_merged_df['created_at'].max()
min_date_metrics = metrics_merged_df['created_at'].min()
print(f"metrics_merged_df - Date Min: {min_date_metrics}, Date Max: {max_date_metrics}")

max_date_job_events = job_events_merged_df['received_at'].max()
min_date_job_events = job_events_merged_df['received_at'].min()
print(f"job_events_merged_df - Date Min: {min_date_job_events}, Date Max: {max_date_job_events}")

max_date_jobs = jobs_merged_df['started_at'].max()
min_date_jobs = jobs_merged_df['started_at'].min()
print(f"jobs_merged_df - Date Min: {min_date_jobs}, Date Max: {max_date_jobs}")


metrics_merged_df - Date Min: 2022-11-02 08:43:45.619000, Date Max: 2023-11-15 14:18:24.413000
job_events_merged_df - Date Min: 2022-02-22 09:43:18.114000, Date Max: 2023-11-16 12:53:08.928000
jobs_merged_df - Date Min: 2022-02-22 09:43:18.116000, Date Max: 2023-11-15 14:10:34.430000


-----------------------------------------------